In [23]:
# Importing all of the required python packages
import sys
import os

In [ ]:
# Setting up the folder paths.
sys.path.append('./lib')

In [25]:
# Import resuable code (scripts) within .py files.
# IMPORTANT - restart the kerneal when new functions are added to the .py lib files.
## If this is not DO - Python loads a module once and doesn’t automatically reload it when you change the file.

# import data_wrangle # for debugging - comment out when done.
# dir(data_wrangle) # for debugging - comment out when done.
from data_wrangle import load_csv, remove_cols

In [27]:
df = load_csv('./data/raw/U.S._Chronic_Disease_Indicators.csv')
print(df.head())

   YearStart  YearEnd LocationAbbr   LocationDesc     DataSource  \
0       2020     2020           US  United States          BRFSS   
1       2015     2019           AR       Arkansas  US Cancer DVT   
2       2015     2019           CA     California  US Cancer DVT   
3       2015     2019           CO       Colorado  US Cancer DVT   
4       2015     2019           GA        Georgia  US Cancer DVT   

           Topic                                           Question  Response  \
0  Health Status            Recent activity limitation among adults       NaN   
1         Cancer    Invasive cancer (all sites combined), incidence       NaN   
2         Cancer  Cervical cancer mortality among all females, u...       NaN   
3         Cancer    Invasive cancer (all sites combined), incidence       NaN   
4         Cancer  Prostate cancer mortality among all males, und...       NaN   

  DataValueUnit      DataValueType  ...  TopicID  QuestionID ResponseID  \
0        Number  Age-adjusted

In [28]:
df2 = load_csv('./data/raw/US_Census_Data_2022_v01.csv')
print(df2.head())

                 Label (Grouping) Alabama!!Total population!!Estimate  \
0  TOTAL NUMBER OF RACES REPORTED                                 NaN   
1                Total population                           5,074,296   
2                        One race                               94.9%   
3                       Two races                                4.7%   
4                     Three races                                0.3%   

  Alabama!!Total population!!Margin of Error  \
0                                        NaN   
1                                      *****   
2                                       ±0.2   
3                                       ±0.2   
4                                       ±0.1   

  Alaska!!Total population!!Estimate  \
0                                NaN   
1                            733,583   
2                              85.8%   
3                              12.8%   
4                               1.3%   

  Alaska!!Total population!!Mar

In [29]:
clean_df = remove_cols(df)
print(clean_df.head())

   YearStart  YearEnd LocationAbbr   LocationDesc     DataSource  ... TopicID  \
0       2020     2020           US  United States          BRFSS  ...     HEA   
1       2015     2019           AR       Arkansas  US Cancer DVT  ...     CAN   
2       2015     2019           CA     California  US Cancer DVT  ...     CAN   
3       2015     2019           CO       Colorado  US Cancer DVT  ...     CAN   
4       2015     2019           GA        Georgia  US Cancer DVT  ...     CAN   

  QuestionID DataValueTypeID StratificationCategoryID1  StratificationID1  
0      HEA04      AGEADJMEAN                       SEX               SEXF  
1      CAN07            NMBR                       SEX               SEXM  
2      CAN03            NMBR                   OVERALL                OVR  
3      CAN07            NMBR                      RACE                HIS  
4      CAN05            NMBR                      RACE                WHT  

[5 rows x 24 columns]
